In [1]:
import Pkg; Pkg.activate(".")
push!(LOAD_PATH, "src/");

In [2]:
using BenchmarkTools
using ProgressMeter
using PyPlot
using Random
using Santa

In [3]:
cities = read_cities("../input/cities.csv")
path = read_path(cities, "/home/maxmouchet/Downloads/1516917.csv")
verify!(path)
score(path)

1.5169178997302188e6

In [246]:
# Assume that everything is reversed except the boundaries
@inbounds function score_reverse(path; start=1)
    dist = 0.0

    if (start % 10 == 0) && !path[1].p
        dist += distance(path[1], path[end-1])*1.1
    else
        dist += distance(path[1], path[end-1])
    end
    
    if ((length(path)+start-2) % 10 == 0) && !path[2].p
        dist += distance(path[2], path[end])*1.1
    else
        dist += distance(path[2], path[end])
    end
    
    for i in 2:length(path)-2
        if ((i+(start-1)) % 10 == 0) && !path[length(path)-i+1].p
            dist += distance(path[length(path)-i+1], path[length(path)-i])*1.1
        else
            dist += distance(path[length(path)-i+1], path[length(path)-i])
        end
    end

    dist
end

score(path), score_reverse(reverse(path))

(1.5169178997302188e6, 1.5169178997302188e6)

In [209]:
rev_path = reverse(path);
@time score(path);
@time score(rev_path);

  0.000878 seconds (7 allocations: 224 bytes)
  0.000730 seconds (7 allocations: 224 bytes)


In [247]:
function nn_opt(init_path::Vector{City}, start::Int, K::Int)
    path = copy(init_path)
    best = score(path, start=start)
    println(best)
    for i = 2:length(path)-1 # Boundaries are left untouched
        # i % 1000 == 0 && println("$(i)/$(length(path))")
        bv, bj = best, 0
        for (_, j) in find_closest_cities(path, path[i], K)
            # Protect boundaries
            ((j == 1) || (j == length(path))) && continue
            # ------------------
            #s = score_reverse(view(path, min(i,j)-1:max(i,j)+1), start=start)
            reverse!(path, min(i,j), max(i,j))
            s  = score(path, start=start)
            if s < bv
                bv, bj = s, j
            end
            reverse!(path, min(i,j), max(i,j))
        end
        if bj != 0
            reverse!(path,  min(i,bj), max(i,bj))
            best = bv
            println(best)
        end
    end
    path
end

nn_opt (generic function with 1 method)

In [257]:
nn_opt(path[1:1000], 1, 10);

7939.053797092918
7938.313708129569
7936.431158335324


In [235]:
function nn_opt_v2(init_path::Vector{City}, start::Int, K::Int)
    path = copy(init_path)
    best = score(path, start=start)
    println(best)
    for i = 2:length(path)-1 # Boundaries are left untouched
        # i % 1000 == 0 && println("$(i)/$(length(path))")
        bv, bj = maxintfloat, 0
        for (_, j) in find_closest_cities(path, path[i], K)
            # Protect boundaries
            ((j == 1) || (j == length(path))) && continue
            # ------------------
            s = score_reverse(view(path, min(i,j)-1:max(i,j)+1), start=start)
            #reverse!(path, min(i,j), max(i,j))
            #s  = score(path, start=start)
            if s < bv
                bv, bj = s, j
            end
            #reverse!(path, min(i,j), max(i,j))
        end
        if bj != 0
            reverse!(path,  min(i,bj), max(i,bj))
            best = bv
            # TMP, remove new score
            println(score(path))
        end
    end
    path
end

nn_opt (generic function with 1 method)

In [237]:
@time nn_opt(path[1:100], 1, 10);

  0.000223 seconds (2.44 k allocations: 139.219 KiB)


In [238]:
@benchmark nn_opt(path[1:200], 1, 50)

BenchmarkTools.Trial: 
  memory estimate:  1.06 MiB
  allocs estimate:  21107
  --------------
  minimum time:     1.672 ms (0.00% GC)
  median time:      1.688 ms (0.00% GC)
  mean time:        1.805 ms (5.72% GC)
  maximum time:     5.186 ms (52.31% GC)
  --------------
  samples:          2768
  evals/sample:     1

In [234]:
@benchmark nn_opt(path[1:200], 1, 50)

BenchmarkTools.Trial: 
  memory estimate:  473.75 KiB
  allocs estimate:  1389
  --------------
  minimum time:     4.507 ms (0.00% GC)
  median time:      4.535 ms (0.00% GC)
  mean time:        4.608 ms (0.56% GC)
  maximum time:     9.378 ms (0.00% GC)
  --------------
  samples:          1085
  evals/sample:     1

In [223]:
@benchmark nn_opt(path[1:100], 1, 10)

BenchmarkTools.Trial: 
  memory estimate:  78.50 KiB
  allocs estimate:  493
  --------------
  minimum time:     326.103 μs (0.00% GC)
  median time:      330.040 μs (0.00% GC)
  mean time:        340.696 μs (1.92% GC)
  maximum time:     2.368 ms (85.26% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [38]:
@time nn_opt(path[1:100], 1, 10);

  0.000649 seconds (497 allocations: 78.656 KiB)


In [188]:
a

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [194]:
view(a, 1:3)

3-element view(::Array{Int64,1}, 1:3) with eltype Int64:
 1
 2
 3

In [90]:
function nn_opt(init_path::Vector{City}, indices::Vector{Int}, K::Int)
    path = copy(init_path)
    best = score(path)
    @showprogress for (zzz, i) in enumerate(indices)
        #zzz % 10 == 0 && print("\33[2K [$K-NN] $(zzz)/$(length(indices)) score = $(best)\r")
        bv, bj = best, 0
        for (_, j) in find_closest_cities(path, path[i], K)
            path[j].i == 0 && continue
            reverse!(path, min(i,j), max(i,j))
            s  = score(path)
            if s < bv
                bv, bj = s, j
            end
            reverse!(path, min(i,j), max(i,j))
        end
        if bj != 0
            reverse!(path,  min(i,bj), max(i,bj))
            best = bv
            println(best)
            end# 
    end
    path
end

nn_opt (generic function with 1 method)

In [91]:
cities = read_cities("../input/cities.csv")
path = read_path(cities, "1516773.csv")
score(path)

1.5167739447755208e6

In [14]:
using Distributed

In [21]:
nprocs()
n = 4

4

In [85]:
# Returns k chunks of a vector of size n
function get_chunks(k::Int, n::Int)
    idxs = collect(1:ceil(Int,n/k):n)
    push!(idxs, n+1)
    chunks = []
    for i = 1:length(idxs)-1
        push!(chunks, idxs[i]:idxs[i+1]-1)
    end
    chunks
end

get_chunks (generic function with 1 method)

In [92]:
chunks = get_chunks(5, length(path))

5-element Array{Any,1}:
 1:39554      
 39555:79108  
 79109:118662 
 118663:158216
 158217:197770

In [94]:
nn_opt(path, collect(chunks[2]), 10)

Progress:   8%|███                                      |  ETA: 0:03:52

InterruptException: InterruptException:

In [45]:
idxs = 1:ceil(Int,length(path)/4):length(path)
for i = 1:length(idxs)-1
    println(idxs[i]:idxs[i+1]-1)
end

1:49443
49444:98886
98887:148329
148330:197772


In [46]:
length(path)

197770

In [ ]:
function nn_opt(init_path::Vector{City}, indices::Vector{Int}, K::Int)
    path = copy(init_path)
    best = score(path)
    @showprogress for (zzz, i) in enumerate(indices)
        #zzz % 10 == 0 && print("\33[2K [$K-NN] $(zzz)/$(length(indices)) score = $(best)\r")
        bv, bj = best, 0
        for (_, j) in find_closest_cities(path, path[i], K)
            path[j].i == 0 && continue
            reverse!(path, min(i,j), max(i,j))
            s  = score(path)
            if s < bv
                bv, bj = s, j
            end
            reverse!(path, min(i,j), max(i,j))
        end
        if bj != 0
            reverse!(path,  min(i,bj), max(i,bj))
            best = bv
            println(best)
            end# 
    end
    path
end

In [12]:
while true
    res = nn_opt(path, collect(10:10:length(path)-1), 50);
    if res == path
        println("done")
        break
    end
    path = res
    println(score(path))
end

Progress:   8%|███                                      |  ETA: 0:07:23

InterruptException: InterruptException:

In [53]:
new_path = nn_opt(path, collect(2:100), 300);

Progress:   2%|█                                        |  ETA: 0:00:18

1.5173894154632282e6


Progress:   3%|█                                        |  ETA: 0:00:18

1.5173736506408183e6


Progress:   4%|██                                       |  ETA: 0:00:17

...

Progress:  47%|███████████████████                      |  ETA: 0:00:11

1.516841664703921e6


Progress:  48%|████████████████████                     |  ETA: 0:00:11

1.5168377592373076e6


Progress: 100%|█████████████████████████████████████████| Time: 0:00:23


In [57]:
new_path = nn_opt(new_path, collect(2:100), 300);

Progress:  14%|██████                                   |  ETA: 0:00:16

1.5167861018011188e6


Progress: 100%|█████████████████████████████████████████| Time: 0:00:23


In [14]:
score(new_path), verify!(new_path)
out = vcat("Path", map(c -> c.i, new_path))
write("1516773.csv", join(out, "\n"))

1273279